In [ ]:
import numpy as np
import random
import time
from IPython.display import clear_output

class TownBoard:
    def __init__(self, size=20, num_houses=10):
        self.size = size
        self.num_houses = num_houses
        self.grid = np.zeros((size, size), dtype=int)
        self.cell_types = {
            0: {'name': 'tree', 'emoji': '🌲', 'flammable': True},
            1: {'name': 'house', 'emoji': '🏠', 'flammable': True},
            2: {'name': 'fire', 'emoji': '🔥', 'flammable': False},
            3: {'name': 'burned', 'emoji': '🪵', 'flammable': False},
            4: {'name': 'road', 'emoji': '⬛', 'flammable': False},
            5: {'name': 'tower', 'emoji': '🗼', 'flammable': False},
            6: {'name': 'car', 'emoji': '🚗', 'flammable': False}
        }
        self.warning_active = False
        self.house_paths = []
        self.cars = []
        self.car_times = []

    def generate_town(self):
        self.grid.fill(0)
        center = self.size // 2
        self.grid[center][center] = 5
        self.house_positions = []
        placed = 0
        while placed < self.num_houses:
            x, y = random.randint(1, self.size - 2), random.randint(1, self.size - 2)
            if self.grid[y][x] == 0:
                self.grid[y][x] = 1
                path = self._connect_to_edge(x, y)
                if path:
                    self.house_positions.append((x, y))
                    self.house_paths.append(path)
                    placed += 1

    def _connect_to_edge(self, x, y):
        edges = [(0, y), (self.size - 1, y), (x, 0), (x, self.size - 1)]
        ex, ey = min(edges, key=lambda pos: abs(pos[0] - x) + abs(pos[1] - y))
        path = []
        cx, cy = x, y
        while cx != ex:
            cx += 1 if ex > cx else -1
            if self.grid[cy][cx] == 0:
                self.grid[cy][cx] = 4
            path.append((cx, cy))
        while cy != ey:
            cy += 1 if ey > cy else -1
            if self.grid[cy][cx] == 0:
                self.grid[cy][cx] = 4
            path.append((cx, cy))
        return path

    def assign_cars(self, fire_positions):
        for path in self.house_paths:
            if not path:
                continue
            start = path[0]
            x, y = start
            delay = 0
            self.cars.append([x, y, path[:], delay, 0])  # (x, y, path, delay, steps)

    def move_cars(self):
        updated_cars = []
        for x, y, path, delay, steps in self.cars:
            if delay > 0:
                updated_cars.append([x, y, path, delay - 1, steps + 1])
                continue
            self.grid[y][x] = 4  # Leave behind a road
            if path:
                x, y = path.pop(0)
                if 0 <= x < self.size and 0 <= y < self.size:
                    self.grid[y][x] = 6
                    updated_cars.append([x, y, path, 0, steps + 1])
            else:
                self.car_times.append(steps)
        self.cars = updated_cars

    def display(self):
        clear_output(wait=True)
        if self.warning_active:
            print("\n🚨 FOREST FIRE WARNING 🚨\n🚨 WEE-OO WEE-OO 🚨\n")
        for row in self.grid:
            print(' '.join(self.cell_types[cell]['emoji'] for cell in row))
        print("\nLegend: 🌲 Tree | 🏠 House | 🔥 Fire | 🪵 Burned | ⬛ Road | 🗼 Tower | 🚗 Car")

class FireSimulator:
    def __init__(self, board):
        self.board = board
        self.burning_cells = set()
        self.time_step = 0
        self.warning_triggered = False
        self.evacuated_reported = False

    def ignite_random(self, count=2):
        self.fire_origins = []
        for _ in range(count):
            while True:
                x, y = random.randint(0, self.board.size-1), random.randint(0, self.board.size-1)
                if self.board.grid[y][x] in [0, 1]:
                    self._ignite_cell(x, y)
                    self.fire_origins.append((x, y))
                    break

    def _ignite_cell(self, x, y):
        self.board.grid[y][x] = 2
        self.burning_cells.add((x, y))
        if not self.warning_triggered:
            self.board.warning_active = True
            self.board.assign_cars(self.fire_origins)
            self.warning_triggered = True

    def spread_fire(self):
        new_fires = set()
        for (x, y) in self.burning_cells:
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < self.board.size and 0 <= ny < self.board.size:
                        if self.board.grid[ny][nx] in [0, 1] and random.random() < 0.6:
                            new_fires.add((nx, ny))
        for x, y in self.burning_cells:
            self.board.grid[y][x] = 3
        for x, y in new_fires:
            self._ignite_cell(x, y)
        self.burning_cells = new_fires
        self.time_step += 1
        return len(new_fires)

    def simulate(self, steps=50, delay=0.8):
        self.ignite_random()
        for _ in range(steps):
            self.board.display()
            print(f"Time Step: {self.time_step} | Fires: {len(self.burning_cells)}")
            new_fires = self.spread_fire()
            self.board.move_cars()
            print(f"New fires started: {new_fires}")
            time.sleep(delay)

            if not self.board.cars and not self.evacuated_reported:
                print("\n🚗 All vehicles evacuated!")
                for i, t in enumerate(self.board.car_times):
                    print(f"Car {i+1} evacuated in {t} minutes")
                if self.board.car_times:
                    print(f"Total evacuation time: {max(self.board.car_times)} minutes")
                self.evacuated_reported = True
                input("\nPress Enter to continue the simulation...")

        if self.burning_cells:
            print("\n🔥 Fire still burning after full simulation.")
        else:
            print("\n🔥 Fire burned out!")

# Run the simulation
town = TownBoard(size=20, num_houses=12)
town.generate_town()
sim = FireSimulator(town)
sim.simulate(steps=20, delay=0.8)
